# 任务描述

这次的任务就是实现一个seq2seq 机器翻译模型。基本的RNN seq2seq 模型已经给我们实现好了。我们任务就是在这基本的RNN模型上边更加进步，我这次就是实现了一个GRU 的seq2seq 模型 并且也使用了课上介绍的Bahdanau Attention 来获得一个更高的Bleu score

# 数据准备 （Data Preparation）

In [31]:
from sacrebleu.metrics import BLEU
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
import argparse, time
import numpy as np

from torch import Tensor
import random


# 加载数据 (to load the data)
def load_data(num_train):
    zh_sents = {}
    en_sents = {}
    for split in ['train', 'val', 'test']:
        zh_sents[split] = []
        en_sents[split] = []
        with open(f"./data/zh_en_{split}.txt", encoding='utf-8') as f:
            for line in f.readlines():
                zh, en = line.strip().split("\t")
                zh = zh.split()
                en = en.split()
                zh_sents[split].append(zh)
                en_sents[split].append(en)
    num_train = len(zh_sents['train']) if num_train==-1 else num_train
    zh_sents['train'] = zh_sents['train'][:num_train]
    en_sents['train'] = en_sents['train'][:num_train]
    print("训练集 验证集 测试集大小分别为", len(zh_sents['train']), len(zh_sents['val']), len(zh_sents['test']))
    return zh_sents, en_sents

# 构建词表 (to build the vocabulary, map word to index)
class Vocab():
    def __init__(self):
        self.word2idx = {}
        self.word2cnt = {}
        self.idx2word = []
        self.add_word("[BOS]")
        self.add_word("[EOS]")
        self.add_word("[UNK]")
        self.add_word("[PAD]")
    
    def add_word(self, word):
        """
        将单词word加入到词表中
        """
        if word not in self.word2idx:
            self.word2cnt[word] = 1
            self.word2idx[word] = len(self.idx2word)
            self.idx2word.append(word)
        self.word2cnt[word] += 1
    
    def add_sent(self, sent):
        """
        将句子sent中的每一个单词加入到词表中
        sent是由单词构成的list
        """
        for word in sent:
            self.add_word(word)
    
    def index(self, word):
        """
        若word在词表中则返回其下标，否则返回[UNK]对应序号
        """
        #return index of the word else return index of unknown if word not in dictionary
        return self.word2idx.get(word, self.word2idx["[UNK]"])
    
    def encode(self, sent, max_len):
        """
        在句子sent的首尾分别添加BOS和EOS之后编码为整数序列
        """
        #returns the index of sentence in dictionary
        encoded = [self.word2idx["[BOS]"]] + [self.index(word) for word in sent][:max_len] + [self.word2idx["[EOS]"]]
        return encoded
    
    def decode(self, encoded, strip_bos_eos_pad=False):
        """
        将整数序列解码为单词序列
        """
        #returns the word of sentence in dictionary
        return [self.idx2word[_] for _ in encoded if not strip_bos_eos_pad or self.idx2word[_] not in ["[BOS]", "[EOS]", "[PAD]"]]
    
    def __len__(self):
        """
        返回词表大小
        """
        return len(self.idx2word)

# 模型原理介绍

序列到序列模型（Sequence-to-Sequence, seq2seq）是一种广泛应用于自然语言处理任务（如机器翻译、文本摘要等）的深度学习架构。其核心由两个部分组成：编码器（Encoder）和解码器（Decoder）。

编码器的作用是将输入序列（如句子）转化为一个固定长度的上下文向量（Context Vector），表示输入信息的抽象特征。该编码器通常由RNN（循环神经网络）、LSTM（长短期记忆网络）或GRU（门控循环单元）构成。

## GRU Cell

本任务是以GRU来创造encoder 的。GRU（门控循环单元）编码器的作用是将输入序列转换为一个固定长度的向量表示，通常称为上下文向量（Context Vector）。这个上下文向量是对整个输入序列的关键特征和依赖关系的压缩表示。GRU通过逐步处理序列数据并更新隐藏状态（Hidden State）来完成这一任务。

In [32]:
class GRUCell(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        #Gates: Update and Reset
        self.weight_ih = nn.Linear(input_size, 2 * hidden_size)
        self.weight_hh = nn.Linear(hidden_size, 2 * hidden_size)

        #Candidate hidden state
        self.weight_candidate = nn.Linear(input_size, hidden_size)
        self.weight_hh_candidate = nn.Linear(hidden_size, hidden_size)

    def forward(self, input, hx):
        """
        input: (batch_size, input_size)
        hx: (batch_size, hidden_size)
        """
        # Compute gates
        gates = self.weight_ih(input) + self.weight_hh(hx)
        z_t, r_t = torch.split(torch.sigmoid(gates), self.hidden_size, dim=1)  # Split into update/reset gates
        
        # Compute candidate hidden state
        candidate = torch.tanh(self.weight_candidate(input) + r_t * self.weight_hh_candidate(hx))
        
        # Update hidden state
        hx = (1 - z_t) * hx + z_t * candidate
        return hx

# Encoder (编码器)

在此使用编程过的GRU Cell，带入我们的Encoder, 会有这些的好处：

缓解梯度消失问题：GRU通过门控机制动态控制隐藏状态的更新，使得梯度在反向传播时能够更好地传递。

捕捉长短期依赖：GRU中的更新门能够决定保留多少历史信息和引入多少新信息，这种动态调整使得GRU能够同时记住短期和长期的依赖关系。

提高训练稳定性：GRU的门控机制使得模型在训练时更稳定，收敛更快，尤其是在数据量有限的情况下。

适应不同长度的输入序列：GRU能够根据输入序列长度动态调整记忆机制，从而在短序列和长序列任务中都能表现出色。


In [33]:
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(EncoderRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_size = hidden_size
        self.gru = GRUCell(embedding_dim, hidden_size)
        
    
    def forward(self, input, hidden):
        """
        input: N
        hidden: N * H
        
        输出更新后的隐状态hidden（大小为N * H）
        """
        # seq_len, batch_size = input_seq.size()
        # outputs = torch.zeros(seq_len, batch_size, self.hidden_size, device=input_seq.device)

        # embedded = self.embedding(input)

        # for t in range(seq_len):
        #     hidden = self.gru(embedded[t], hidden)
        #     outputs[t] = hidden
        # return outputs, hidden
        embedded = self.embedding(input)
        hidden = self.gru(embedded, hidden)
        return hidden

# Attention Mechanism （注意力机制）

解码器的作用是利用编码器生成的上下文向量，逐步生成输出序列。传统的seq2seq模型可能因为上下文向量过于简化而导致信息丢失。为了提升模型性能，我在解码器中引入了Bahdanau注意力机制。

Bahdanau注意力机制通过动态地关注输入序列的不同部分，分配不同的权重，使得解码器在生成每个输出词时可以参考输入序列的相关部分。这样解决了固定长度上下文向量无法充分表达长序列信息的问题。


我会选择Bahdanau Attention的原因是因为它比传统注意力RNN更好 更合适：

1. 更好地处理长序列
对于长输入序列，传统注意力方法往往无法有效关注所有信息，因为权重分布可能失衡。
Bahdanau注意力机制通过在每个解码步骤重新计算权重，确保即使是长距离依赖的信息也能被充分考虑。

2. 解耦编码器与解码器上下文
传统方法通常依赖一个固定的上下文向量（context vector）来概括整个输入序列，这种方法会丢失部分信息，尤其是长序列中的细节。
Bahdanau注意力机制通过动态加权计算多个上下文向量，避免了固定上下文向量的局限性，从而保留了输入序列的更多信息

In [34]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        self.W = nn.Linear(hidden_size, hidden_size)
        self.U = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1)
    
    def forward(self, hidden, encoder_outputs):
        """
        hidden: N * H
        encoder_outputs: S * N * H
        
        输出attention加权后的context（大小为N * H）
        """
        seq_len, batch_size, hidden_size = encoder_outputs.size()
        hidden = hidden.unsqueeze(0).expand(seq_len, -1, -1)  # S * N * H
        score = self.v(torch.tanh(self.W(hidden) + self.U(encoder_outputs)))  # S * N * 1
        attention_weights = F.softmax(score, dim=0)  # S * N * 1
        context = torch.sum(attention_weights * encoder_outputs, dim=0)  # N * H
        return context, attention_weights

# Decoder （解码器）

在此把我们之前编码的Bahdanau 注意力机制带进我们的Decoder， 使得我们的解码器比传统解码器更加有优势比如：

1. 动态关注：每一步解码时都能动态选择输入序列中的关键部分。
2. 长序列处理能力更强：解决固定上下文向量导致的信息丢失问题。
3. 梯度流动更好：训练更稳定，尤其在长序列任务中效果显著。
4. 模型更可解释：注意力分布提供直观的输入与输出对应关系。
5. 提高输出质量：生成更流畅、准确的序列。

In [35]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, dropout = 0.1):
        super(DecoderRNN, self).__init__()
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_size = hidden_size
        self.gru = GRUCell(embedding_dim, hidden_size)

        #add dropout
        self.dropout = nn.Dropout(dropout)

        #add layer normalization
        self.layer_norm = nn.LayerNorm(hidden_size)
    
        # self.h2o = nn.Linear(hidden_size, vocab_size) //replaced with self.out
        self.softmax = nn.LogSoftmax(dim=1)

        self.attention = BahdanauAttention(self.hidden_size)
        self.attention_combine = nn.Linear(hidden_size + embedding_dim, embedding_dim)
        
        #multiple layers with activation
        self.out = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size)
        )
    
    def forward(self, input, hidden, encoder_outputs):
        """
        input: N
        hidden: N * H
        
        输出对于下一个时间片的预测output（大小为N * V）更新后的隐状态hidden（大小为N * H）
        """
        embedding = self.embedding(input)
        embedding = self.dropout(embedding)

        context, attention_weights = self.attention(hidden, encoder_outputs)

        #combine context and embedding
        gru_input = torch.cat((embedding, context), dim=1)
        gru_input = self.attention_combine(gru_input)
        gru_input = F.relu(gru_input)

        #GRU step
        hidden = self.gru(gru_input, hidden)
        hidden = self.layer_norm(hidden)

        #output projection
        output = self.out(hidden)
        output = self.softmax(output)
        return output, hidden, attention_weights

# Label Smoothing Loss (标签平滑损失)

标签平滑（Label Smoothing） 是一种在分类任务中常用的正则化技术，用于缓解模型过拟合和输出过于自信的问题。在训练时，模型通常以独热编码（One-Hot Encoding）的标签作为目标。\

例如，对于一个3类分类问题，如果正确类别是2，目标标签是 [0, 0, 1]. 
这种方式明确地告诉模型，某个类别的概率应该是100%，其他类别的概率是0%。

然而，这种方式可能导致模型：

过度自信（Overconfident）：模型在训练过程中可能倾向于输出接近1的概率值，而这在实际任务中并不总是最优的。
过拟合（Overfitting）：模型可能对训练数据的标签分布过度适应，从而在测试数据上表现较差。

所以根据大语言模型的推荐，我也实现了 label smoothing loss 来增加我的 Bleu score

In [36]:
# Add label smoothing loss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, size, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='batchmean')
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 1))
        true_dist.scatter_(1, target.data.unsqueeze(1), 1.0 - self.smoothing)
        return self.criterion(x, true_dist.detach())

# Seq2Seq Model (全部合并)

In [37]:
class Seq2Seq(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, embedding_dim, hidden_size, max_len):
        super(Seq2Seq, self).__init__()
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.hidden_size = hidden_size
        self.encoder = EncoderRNN(len(src_vocab), embedding_dim, hidden_size)
        self.decoder = DecoderRNN(len(tgt_vocab), embedding_dim, hidden_size)
        self.max_len = max_len
        
    def init_hidden(self, batch_size):
        """
        初始化编码器端隐状态为全0向量（大小为1 * H）
        """
        device = next(self.parameters()).device
        return torch.zeros(batch_size, self.hidden_size).to(device)
    
    def init_tgt_bos(self, batch_size):
        """
        预测时，初始化解码器端输入为[BOS]（大小为batch_size）
        """
        device = next(self.parameters()).device
        return (torch.ones(batch_size)*self.tgt_vocab.index("[BOS]")).long().to(device)
    
    def forward_encoder(self, src):
        """
        src: N * L
        编码器前向传播，输出最终隐状态hidden (N * H)和隐状态序列encoder_hiddens (N * L * H)
        """
        Bs, Ls = src.size()
        hidden = self.init_hidden(batch_size=Bs)
        encoder_hiddens = []
        # 编码器端每个时间片，取出输入单词的下标，与上一个时间片的隐状态一起送入encoder，得到更新后的隐状态，存入enocder_hiddens
        for i in range(Ls):
            input = src[:, i]
            hidden = self.encoder(input, hidden)
            encoder_hiddens.append(hidden)
        encoder_hiddens = torch.stack(encoder_hiddens, dim=0)
        return hidden, encoder_hiddens
    
    def forward_decoder(self, tgt, hidden, encoder_hiddens, tfr):
        """
        tgt: N
        hidden: N * H
        
        解码器前向传播，用于训练，使用teacher forcing，输出预测结果outputs，大小为N * L * V，其中V为目标语言词表大小
        """
        Bs, Lt = tgt.size()
        outputs = []
        input = tgt[:, 0]

        for i in range(Lt):
            output, hidden, attention_weights = self.decoder(input, hidden, encoder_hiddens)
            outputs.append(output)

            use_teacher_forcing = random.random() < tfr
            if use_teacher_forcing and i < Lt-1:
                input = tgt[:, i+1]
            else:
                input = output.argmax(-1)
        outputs = torch.stack(outputs, dim=1)
        return outputs
        
    
    def forward(self, src, tgt, tfr):
        """
            src: 1 * Ls
            tgt: 1 * Lt
            
            训练时的前向传播
        """
        hidden, encoder_hiddens = self.forward_encoder(src)
        outputs = self.forward_decoder(tgt, hidden, encoder_hiddens, tfr)
        return outputs
    
    def predict(self, src):
        """
            src: 1 * Ls
            
            用于预测，解码器端初始输入为[BOS]，之后每个位置的输入为上个时间片预测概率最大的单词
            当解码长度超过self.max_len或预测出了[EOS]时解码终止
            输出预测的单词编号序列，大小为1 * L，L为预测长度
        """
        hidden, encoder_hiddens = self.forward_encoder(src)
        input = self.init_tgt_bos(batch_size=src.shape[0])
        preds = []
        while len(preds) < self.max_len:
            output, hidden, attention_weights = self.decoder(input, hidden, encoder_hiddens)
            input = output.argmax(-1)
            preds.append(input)
            if input == self.tgt_vocab.index("[EOS]"):
                break
        preds = torch.stack(preds, dim=-1)
        return preds

# 上载数据 （Data Loader）

按照之前提供的，没什么改变来上载我们的数据，准备给我们的seq2seq 模型

In [38]:
# 构建Dataloader
def collate(data_list):
    src = torch.stack([torch.LongTensor(_[0]) for _ in data_list])
    tgt = torch.stack([torch.LongTensor(_[1]) for _ in data_list])
    return src, tgt

def padding(inp_ids, max_len, pad_id):
    max_len += 2    # include [BOS] and [EOS]
    ids_ = np.ones(max_len, dtype=np.int32) * pad_id
    max_len = min(len(inp_ids), max_len)
    ids_[:max_len] = inp_ids
    return ids_

def create_dataloader(zh_sents, en_sents, max_len, batch_size, pad_id):
    dataloaders = {}
    for split in ['train', 'val', 'test']:
        shuffle = True if split=='train' else False
        datas = [(padding(zh_vocab.encode(zh, max_len), max_len, pad_id), padding(en_vocab.encode(en, max_len), max_len, pad_id)) for zh, en in zip(zh_sents[split], en_sents[split])]
        dataloaders[split] = torch.utils.data.DataLoader(datas, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders['train'], dataloaders['val'], dataloaders['test']


# 训练模型

开始训练我们的模型。 在此，大部分的编码跟之前的差不多，除了一个部分：teacher forcing 和 之前的 label smoothing loss

Teacher Forcing 是一种在训练序列到序列模型（如翻译或生成任务）时常用的技术。其核心思想是：在每个解码步骤中，使用真实的目标词作为当前时间步的输入，而不是模型在上一时间步生成的预测词。

`TFR=max(0.4, 1.0 − current_epoch/max_epoch​)`

意味着：
在训练初期，TFR 接近 1，更多使用真实目标词（Teacher Forcing）；
随着训练逐渐进行，TFR 减少至 0.4，模型逐渐更多依赖自己的预测进行训练

这也是因为在文件里所说的，如果没有teacher forcing 的时候，在训练早期的输入可能无关，增加错误预测的传播，降低训练效率

In [39]:
# 训练模型
def train_loop(model, optimizer, criterion, loader, device, current_epoch, max_epochs):
    model.train()
    epoch_loss = 0.0

    #add label smoothing
    smoothing = 0.1
    criterion = LabelSmoothingLoss(size=model.decoder.vocab_size, smoothing=smoothing)

    for src, tgt in tqdm(loader):
        src = src.to(device)
        tgt = tgt.to(device)

        tfr = max(0.4, 1.0 - current_epoch/max_epochs)    # teacher forcing ratio
        outputs = model(src, tgt, tfr)
        loss = criterion(outputs[:,:-1,:].reshape(-1, outputs.shape[-1]), tgt[:,1:].reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)     # 裁剪梯度，将梯度范数裁剪为1，使训练更稳定
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(loader)
    return epoch_loss

# 模型评价 （Evaluation）

test_loop 的代码跟之前的一样，没有改变evaluation 的方式

In [40]:
def test_loop(model, loader, tgt_vocab, device):
    model.eval()
    bleu = BLEU(force=True)
    hypotheses, references = [], []
    for src, tgt in tqdm(loader):
        B = len(src)
        for _ in range(B):
            _src = src[_].unsqueeze(0).to(device)     # 1 * L
            with torch.no_grad():
                outputs = model.predict(_src)         # 1 * L
            
            # 保留预测结果，使用词表vocab解码成文本，并删去BOS与EOS
            ref = " ".join(tgt_vocab.decode(tgt[_].tolist(), strip_bos_eos_pad=True))
            hypo = " ".join(tgt_vocab.decode(outputs[0].cpu().tolist(), strip_bos_eos_pad=True))
            references.append(ref)    # 标准答案
            hypotheses.append(hypo)   # 预测结果
    
    score = bleu.corpus_score(hypotheses, [references]).score      # 计算BLEU分数
    return hypotheses, references, score

# 主函数 （main function)

In [46]:
import sys
import argparse



# 主函数
if __name__ == '__main__':
    sys.argv = [
        'ipykernel_launcher.py',  
        '--num_train', '-1',      
        '--max_len', '10',
        '--batch_size', '128',
        '--optim', 'adam',
        '--num_epoch', '10',
        '--lr', '0.0005'
    ]

    parser = argparse.ArgumentParser()      
    parser.add_argument('--num_train', default=-1, type=int, help="训练集大小，等于-1时将包含全部训练数据")
    parser.add_argument('--max_len', default=10, type=int, help="句子最大长度")
    parser.add_argument('--batch_size', default=128, type=int)
    parser.add_argument('--optim', default='adam')
    parser.add_argument('--num_epoch', default=10, type=int)
    parser.add_argument('--lr', default=0.0005, type=float)
    args = parser.parse_args()

    zh_sents, en_sents = load_data(args.num_train)

    zh_vocab = Vocab()
    en_vocab = Vocab()
    for zh, en in zip(zh_sents['train'], en_sents['train']):
        zh_vocab.add_sent(zh)
        en_vocab.add_sent(en)
    print("中文词表大小为", len(zh_vocab))
    print("英语词表大小为", len(en_vocab))

    trainloader, validloader, testloader = create_dataloader(zh_sents, en_sents, args.max_len, args.batch_size, pad_id=zh_vocab.word2idx['[PAD]'])

    torch.manual_seed(1)
    #Use GPU for training
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Using device: ", device)
    if torch.cuda.is_available():
        print("GPU: ", torch.cuda.get_device_name(0))

    #initialise Model
    model = Seq2Seq(zh_vocab, en_vocab, embedding_dim=256, hidden_size=256, max_len=args.max_len)
    model.to(device)
    if args.optim=='sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
    elif args.optim=='adam':
        optimizer = torch.optim.Adam(model.parameters(), lr = args.lr) #introduce more parameters for Adam
    weights = torch.ones(len(en_vocab)).to(device)
    weights[en_vocab.word2idx['[PAD]']] = 0 # set the loss of [PAD] to zero
    criterion = nn.NLLLoss(weight=weights)

    # #introduce scheduler
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer,
    #     mode='max',
    #     factor=0.5,
    #     patience=2,
    #     verbose=True
    # )

    # 训练
    start_time = time.time()
    best_score = 0.0
    best_epoch = 0
    
    for epoch in range(args.num_epoch):
        loss = train_loop(model, optimizer, criterion, trainloader, device, epoch, args.num_epoch)
        hypotheses, references, bleu_score = test_loop(model, validloader, en_vocab, device)
        # scheduler.step(bleu_score)
        # 保存验证集上bleu最高的checkpoint
        if bleu_score > best_score:
            torch.save(model.state_dict(), "model_best.pt")
            best_score = bleu_score
            best_epoch = epoch
        print(f"Epoch {epoch}: loss = {loss}, valid bleu = {bleu_score}")
        print(references[0])
        print(hypotheses[0])
    end_time = time.time()

    #测试
    model.load_state_dict(torch.load("model_best.pt"))
    hypotheses, references, bleu_score = test_loop(model, testloader, en_vocab, device)
    print(f"Test bleu = {bleu_score}")
    print(references[0])
    print(hypotheses[0])
    print(f"Training time: {round((end_time - start_time)/60, 2)}min")

训练集 验证集 测试集大小分别为 26187 1000 1000
中文词表大小为 14718
英语词表大小为 11475
Using device:  cuda
GPU:  NVIDIA GeForce RTX 3050 Ti Laptop GPU


100%|██████████| 8/8 [00:19<00:00,  2.50s/it]


Epoch 0: loss = 3.1951938024381312, valid bleu = 0.7450135118611164
Why are you [UNK] my son?
Do you like to go to the good


100%|██████████| 8/8 [00:18<00:00,  2.36s/it]


Epoch 1: loss = 2.4383464161942645, valid bleu = 3.4159235725936137
Why are you [UNK] my son?
How much do you know me to me?


...

Epoch 8: loss = 1.5274026841652102, valid bleu = 11.486475430260418
Why are you [UNK] my son?
Why did you call me the


100%|██████████| 8/8 [00:21<00:00,  2.70s/it]
C:\Users\wenji\AppData\Local\Temp\ipykernel_27424\2389720481.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_st

Epoch 9: loss = 1.3903224712464868, valid bleu = 13.644033303596743
Why are you [UNK] my son?
Why did you let me the the


100%|██████████| 8/8 [00:26<00:00,  3.31s/it]

Test bleu = 14.456257891945436
We should be safe here.
We should be your right here.
Training time: 9.07min


最终，我们把所有实现的都放在主函数下，并且训练而得到一个很不错的bleu score （~14）

在这里我也尝试了使用optimizer with weight decay 和 scheduler 可是这些导致我的模型训练的结果不符合要求（bleu score 一下子降低到0.2

一下可能是bleu score 降低的原因：
破坏模型的记忆能力
过于频繁地调整学习率
学习率降低过早

所以最终还是去掉了区别

# 实验结果与分析

### 模型性能

1. BLEU分数：模型在测试集上取得了 14.46 的 BLEU 分数，表明模型在句子翻译或生成任务中具备一定的语言建模能力。
BLEU分数在翻译任务中是一种常用的评估指标，分数范围在 0 到 100 之间。我的模型取得的分数显示翻译效果已达到初步可用的水平，但仍有改进空间。

2. 根据readme.md 的参考BLEU，Attention RNN Decoder + GRU 的 BLEU 分数在 13左右，表明翻译结果质量尚可，但在流畅性和准确性上可能还存在一定不足。

### 缺点：

1. 未识别词（[UNK]）的出现反映词表覆盖不足的问题。

2. 句子重复现象（"the the"）和语法不连贯的问题显示模型在复杂句子上的生成能力仍需提升。

3. BLEU 分数虽然不错，但距离高质量翻译任务的水平仍有较大差距。

### 改进方向

1. 数据增强：
增加训练数据的多样性，尤其是包含更丰富词汇和复杂句式的语料。

2. 优化模型结构：
增加词表大小或采用子词分词（如 BPE）减少 [UNK] 的出现。
在解码器中引入更先进的机制（如 Transformer）以提升模型的语义和语法理解能力。